In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [59]:
# Set number of agents and budget
N = 4
B = 1000

In [60]:
# import player data

df = pd.read_csv("player_pts_updated.csv")
df = df.rename(columns={'Total Points': 'True Value', 'std': 'STD', 'Pos': 'Position'})

df.head()

,Player,Position,True Value,STD,ID
0,Patrick Mahomes II,QB,428.4,8.53,0
1,Josh Allen,QB,412.2,10.03,1
2,Jalen Hurts,QB,384.0,11.93,2
3,Joe Burrow,QB,369.0,9.35,3
4,Geno Smith,QB,314.9,6.97,4


In [61]:
# Generating auction subset: take subset of dataset to take only the top players from each position

# create a new dataframe for each position
df_rb = df[df['Position'] == 'RB'].head(N*4)
df_qb = df[df['Position'] == 'QB'].head(N*2)
df_wr = df[df['Position'] == 'WR'].head(N*4)
df_te = df[df['Position'] == 'TE'].head(N*2)
df_k = df[df['Position'] == 'K'].head(N*2)
df_dst = df[df['Position'] == 'DST'].head(N*2)

df_auction = pd.concat([df_qb, df_rb, df_wr, df_k, df_dst, df_te])
# sort df_auction by descending order of true value
df_auction = df_auction.sort_values(by=['True Value'], ascending=False)
df_auction.head()

,Player,Position,True Value,STD,ID
0,Patrick Mahomes II,QB,428.4,8.53,0
1,Josh Allen,QB,412.2,10.03,1
2,Jalen Hurts,QB,384.0,11.93,2
3,Joe Burrow,QB,369.0,9.35,3
4,Geno Smith,QB,314.9,6.97,4


In [62]:
# Convert true values, standard devs to numpy array
trueValues = df_auction['True Value'].to_numpy()
stds = df_auction['STD'].to_numpy()

def value_generator(df):
    '''df --> dict: generates a random value for each player,
        then returns the perceived values as a dictionary'''
    perceivedValues = {}
    for i in range(len(df)):
        perceivedValues[df.iloc[i]['ID']] = np.random.normal(trueValues[i], stds[i])
    return perceivedValues

In [63]:
# Define fair values for each position

v_qb = df_qb['True Value'].sum() / (N)
v_rb = df_rb['True Value'].sum() / (N)
v_wr = df_wr['True Value'].sum() / (N)
v_te = df_te['True Value'].sum() / (N)
v_k = df_k['True Value'].sum() / (N)
v_dst = df_dst['True Value'].sum() / (N)

V = v_qb + v_rb + v_wr + v_te + v_k + v_dst

In [64]:
# Actual auction simulation

# generate values for each player
player_values = [value_generator(df_auction) for _ in range(N)]
player_selections = [[] for _ in range(N)]

# keep track of how many of each position each player selected
player_positions = [[] for _ in range(N)]
budgets = [1000] * N
quotas = {"QB": 2, "RB": 4, "WR" : 4, "TE" : 2, "K": 2, "DST": 2}

# calculating naive bid
def calc_bid(agent, player_id):
    return min((player_values[agent][player_id])/V * B, budgets[agent])

# calculating non-standard deviation strategic bid
def calc_strat_bid(agent, player_id, position, i, df):
    # calculate opportunity cost as the difference between the value of the player and the value of the next best player
    # find the next best player with position "position" after row i in df
    next_best_df = df[(df['Position'] == position) & (df['True Value'] < df.iloc[i]['True Value'])]
    # if next_best_df is empty: return value of player
    if next_best_df.empty:
        return calc_bid(agent, player_id)
    else:
        strategic_value = min(3*(df.iloc[i]['True Value']- next_best_df.iloc[0]['True Value']), budgets[agent])
        return strategic_value
    
# calculating standard deviation strategic bid
def calc_strat_new(agent, player_id, position, i, df):
    # get sub_dataframe of df of after row i
    sub_df = df.iloc[i:]
    # calculate standard deviations of true_values for each position in sub_df
    stds = sub_df.groupby('Position')['True Value'].std(ddof=0)
    prop = stds[position] / sum(stds)

    pos_df = sub_df[sub_df['Position'] == position]
    # if pos_df is empty: return value of player
    if pos_df.empty:
        return calc_bid(agent, player_id)
    else:
        strategic_value = min(prop*budgets[agent], budgets[agent])
        return strategic_value

# keeping track of current row in df_auction
cur_row = 0

# set of strategic agents
strat_agents = [0]

# iterate through df_auction
for row in df_auction.iterrows():
    pos = row[1]['Position']
    ids = row[1]['ID']
    bids = [0]*N # keep track of bids
    for i in range(N): # calculate bid for each player
        # if count of position in player_positions in greater than allotted number of players, then bid 0
        if player_positions[i].count(pos) >= quotas[pos]:
            bids[i] = -1
        else:
            if i in strat_agents:
                bids[i] = calc_strat_new(i, ids, pos, cur_row, df_auction)
            else:
                bids[i] = calc_bid(i, ids) # from paper: min of budget left & fair price

    # if no one can bid, goes to the one who nominated
    if sum(bids) == -4:
        winner = cur_row % 4
    else:
        winner = np.argmax(bids) # select player with highest bid
    # add player to player_selections
    print("Bids:", bids)
    # print("bid legnth", len(bids))
    # print(winner, row[1]['Player ID'])
    player_selections[winner].append(ids) # add selection to player_selections
    player_positions[winner].append(pos) # add position to player_positions
    budgets[winner] -= bids[winner] # add bid to budget
    cur_row += 1

    print("Player", winner, "selected", ids, "with bid", bids[winner], "and budget", budgets[winner])
    print("Drafted Player in Position", pos, "of value", player_values[winner][ids])
    print("")

Bids: [270.94631304148254, 138.07184376704512, 131.55452140390062, 139.3468779711543]
Player 0 selected 0 with bid 270.94631304148254 and budget 729.0536869585175
Drafted Player in Position QB of value 417.6199958724591

Bids: [177.14338340119312, 132.37104020188957, 130.8301891683795, 128.14610806976782]
Player 0 selected 1 with bid 177.14338340119312 and budget 551.9103035573244
Drafted Player in Position QB of value 402.6531338388545

Bids: [-1, 113.67701172629522, 122.94347556611497, 123.06257563085347]
Player 3 selected 2 with bid 123.06257563085347 and budget 876.9374243691466
Drafted Player in Position QB of value 389.7853254887745

Bids: [-1, 111.28385498330358, 118.1298837527753, 125.03663155859915]
Player 3 selected 3 with bid 125.03663155859915 and budget 751.9007928105474
Drafted Player in Position QB of value 396.037900882918

Bids: [-1, 101.26536304280152, 98.4787684660717, -1]
Player 1 selected 4 with bid 101.26536304280152 and budget 898.7346369571985
Drafted Player in 

In [65]:
# print results

for (i, selection) in enumerate(player_selections):
    print("Player", i, "selections:", selection)

print("")

for (i, position) in enumerate(player_positions):
    print("Player", i, "positions:", position)

Player 0 selections: [0, 1, 12, 13, 19, 32, 46, 48, 70, 73, 74, 77, 76, 84, 101, 122]
Player 1 selections: [4, 5, 15, 16, 26, 29, 34, 37, 63, 64, 71, 69, 78, 89, 95, 177]
Player 2 selections: [6, 7, 14, 25, 28, 30, 35, 40, 44, 56, 59, 60, 61, 72, 110, 168]
Player 3 selections: [2, 3, 22, 24, 38, 39, 42, 45, 49, 50, 55, 57, 58, 68, 132, 187]

Player 0 positions: ['QB', 'QB', 'RB', 'RB', 'WR', 'TE', 'RB', 'RB', 'WR', 'WR', 'WR', 'K', 'K', 'TE', 'DST', 'DST']
Player 1 positions: ['QB', 'QB', 'RB', 'RB', 'RB', 'WR', 'RB', 'WR', 'WR', 'WR', 'DST', 'K', 'K', 'DST', 'TE', 'TE']
Player 2 positions: ['QB', 'QB', 'RB', 'WR', 'WR', 'RB', 'RB', 'WR', 'RB', 'DST', 'K', 'WR', 'K', 'DST', 'TE', 'TE']
Player 3 positions: ['QB', 'QB', 'WR', 'RB', 'RB', 'DST', 'RB', 'RB', 'DST', 'WR', 'K', 'K', 'WR', 'WR', 'TE', 'TE']


In [66]:
# print true values for each of the selections
for i in range(N):
    print("Player", i, "selections:")
    print(player_selections[i])
    print([df_auction.loc[df_auction['ID'] == j]['True Value'].values[0] for j in player_selections[i]])
    print("Total value:", sum(df_auction.loc[df_auction['ID'].isin(player_selections[i])]['True Value'].values))

Player 0 selections:
[0, 1, 12, 13, 19, 32, 46, 48, 70, 73, 74, 77, 76, 84, 101, 122]
[428.4, 412.2, 275.3, 271.4, 240.7, 206.3, 175.5, 170.1, 152.0, 149.8, 148.4, 146.0, 146.0, 140.5, 134.0, 121.0]
Total value: 3317.6
Player 1 selections:
[4, 5, 15, 16, 26, 29, 34, 37, 63, 64, 71, 69, 78, 89, 95, 177]
[314.9, 307.0, 265.7, 254.4, 213.9, 211.2, 198.8, 194.6, 157.9, 155.4, 152.0, 152.0, 146.0, 139.0, 136.8, 97.3]
Total value: 3096.9000000000005
Player 2 selections:
[6, 7, 14, 25, 28, 30, 35, 40, 44, 56, 59, 60, 61, 72, 110, 168]
[305.6, 303.6, 269.8, 222.2, 211.6, 209.8, 196.7, 184.2, 180.7, 163.0, 161.0, 159.6, 159.0, 151.0, 129.4, 103.9]
Total value: 3111.1
Player 3 selections:
[2, 3, 22, 24, 38, 39, 42, 45, 49, 50, 55, 57, 58, 68, 132, 187]
[384.0, 369.0, 235.5, 227.0, 189.6, 186.0, 182.5, 180.1, 170.0, 169.0, 164.0, 162.0, 161.6, 153.3, 117.5, 92.8]
Total value: 3143.9


In [67]:
# calculate social welfare using normalized values

# sum of all true values in df_auction
true_value = df_auction['True Value'].sum()

# calculate social welfare for each player as sum of their perceived values divided by the sum of all true values
for player in range(N):
    sum_perceived = [player_values[player][j] for j in player_selections[player]]
    print("Player", player, "social welfare:", sum(sum_perceived)/true_value)

print("Sum of social welfares:", sum([sum([player_values[player][j] for j in player_selections[player]])/true_value for player in range(N)]))

Player 0 social welfare: 0.2586762816408196
Player 1 social welfare: 0.25221873028836345
Player 2 social welfare: 0.2508749508880567
Player 3 social welfare: 0.2547817993170276
Sum of social welfares: 1.0165517621342675


In [68]:
# print the sample standard deviation of the true values for each position
print(df_auction.groupby('Position')['True Value'].std(ddof=0))

Position
DST    19.544820
K       7.348469
QB     48.297863
RB     37.380087
TE     33.983633
WR     31.701338
Name: True Value, dtype: float64
